In [118]:
from datetime import date
from dataclasses import dataclass
import math

In [119]:
MIN_YEAR = 2023
MAX_YEAR = 2033
# -1 pefixes will be used in arrays as days and months are not zero-indexed.
DAYS_IN_24_HR_MONTH         = [-1, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
DAYS_IN_24_HR_MONTH_LEAP_YR = [-1, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
# DAYS_IN_36_HR_MONTH         = [-1, 21, 19, 21, 20, 21, 20, 21, 21, 20, 21, 20, 21]
# DAYS_IN_36_HR_MONTH_LEAP_YR = [-1, 21, 20, 21, 20, 21, 20, 21, 21, 20, 21, 20, 21]
MONTH_ABRVS = [-1, "JA", "FE", "MR", "AR", "MA", "JN", "JL", "AU", "SE", "OC", "NO", "dE"]

In [120]:
def is_leap_year(year):
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

In [121]:
@dataclass
class Month:
    num_days: int
    hours_in_days: list
    month_abrv: str

In [131]:
def get_conversion_definition(year, days_in_months, days_in_monts_leap_yr, conv_hrs_per_day):
    hrs_in_24_hr_month = [-1]
    days_in_24_hr_month = days_in_monts_leap_yr if is_leap_year(year) else days_in_months
    for day in days_in_24_hr_month[1:]:
        hrs_in_24_hr_month.append(day * 24)
    
    year_definition = [-1]
    for month_hrs, month_abrv in zip(hrs_in_24_hr_month[1:], MONTH_ABRVS[1:]):
        num_converted_days = math.floor(month_hrs / conv_hrs_per_day)
        last_day_hrs = month_hrs % conv_hrs_per_day
        hours_in_days = [conv_hrs_per_day for i in range(num_converted_days)]
        if last_day_hrs != 0:
            hours_in_days.append(last_day_hrs)

        year_definition.append(
            Month(num_days=len(hours_in_days),
                  hours_in_days=[-1] + hours_in_days,
                  month_abrv=month_abrv)
        )
        
    return year_definition

In [132]:
class Date24hrs:
    def days_in_month(self, year, month):
        # Return the days in the given month of the given year, accounting for leap years. 
        assert 1 <= month <= 12, month
        if is_leap_year(year):
            return DAYS_IN_24_HR_MONTH_LEAP_YR[month]
        return DAYS_IN_24_HR_MONTH[month]
    
    def check_date(self, year, month, day, hour):
        # Make sure the provided date is valid.
        if not MIN_YEAR <= year <= MAX_YEAR:
            raise ValueError(f'bad! {year=}')
        if not 1 <= month <= 12:
            raise ValueError(f'bad! {month=}')
        if not 1 <= day <= self.days_in_month(year, month):
            raise ValueError(f'bad! {year=} {month=} {day=}')
        if not 0 <= hour <= 23:
            raise ValueError(f'bad! {hour=}')
    
        return year, month, day, hour
        
    def __init__(self, year, month, day, hour):
        self.year, self.month, self.day, self.hour = self.check_date(year, month, day, hour)
        self.day_length = 24
        self.month_abrv = MONTH_ABRVS[self.month]
        
    def to_36_hr_date(self) -> 'Date36hrs':
        # Calculate the total number of hours that have passed from the beginning of the current
        # month until the the current day.
        hrs_elapsed_to_day = self.day * self.day_length
        day_converted = math.floor(hrs_elapsed_to_day / 36)
        # The last day of each 36hr month might not actually have 36hrs in it (a short day).
        # Check if the day is a short day.
        conversion = get_conversion_definition(self.year,
                                               DAYS_IN_24_HR_MONTH,
                                               DAYS_IN_24_HR_MONTH_LEAP_YR,
                                               36)
        short_day = (day_converted == conversion[self.month].num_days - 1) and \
                    (conversion[self.month].hours_in_days[-1] != 36)
        if short_day:
            day_converted += 1
            
        # Total number of hours from the beginning of the current month until the current
        # _hour_ of the current day.
        total_hrs_elapsed = (self.day * self.day_length) + self.hour
        hour_converted = total_hrs_elapsed % conversion[self.month].hours_in_days[day_converted]
        return Date36hrs(self.year, self.month, day_converted, hour_converted)

In [143]:
class Date36hrs:
    def __init__(self, year, month, day, hour):
        self.year = year
        self.month = month
        self.day = day
        self.hour = hour
        self.month_abrv = MONTH_ABRVS[self.month]

In [144]:
epoch = Date24hrs(2023, 4, 9, 1).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=4 (AR), epoch.day=6, epoch.hour=1


In [145]:
epoch = Date24hrs(2023, 4, 29, 12).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=4 (AR), epoch.day=19, epoch.hour=24


In [146]:
epoch = Date24hrs(2024, 2, 29, 23).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2024, epoch.month=2 (FE), epoch.day=20, epoch.hour=11


In [147]:
epoch = Date24hrs(2024, 2, 29, 23).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2024, epoch.month=2 (FE), epoch.day=20, epoch.hour=11


In [148]:
epoch = Date24hrs(2023, 2, 28, 23).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=2 (FE), epoch.day=19, epoch.hour=23


In [149]:
epoch = Date24hrs(2023, 2, 28, 12).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=2 (FE), epoch.day=19, epoch.hour=12
